### **Who are the delayed flights?**
    the definition of flight delay is a flight that did not land on time.
   
    In cases where the flight was delayed at departure and arrived on time or arrived before the estimated landing time -
    this flight will not be considered a delayed flight.


### **Libraries**

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
dataset = pd.read_csv("data.csv")
dataset.head(5)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


### **List of features**
    FL_DATE = Date of the Flight
    OP_CARRIER = Airline Identifier
    OP_CARRIER_FL_NUM = Flight Number
    ORIGIN = Starting Airport Code
    DEST = Destination Airport Code
    CRS_DEP_TIME = Planned Departure Time
    DEP_TIME = Actual Departure Time
    DEP_DELAY = Total Delay on Departure in minutes
    TAXI_OUT = The time duration elapsed between departure from the origin airport gate and wheels off
    WHEELS_OFF = The time point that the aircraft's wheels leave the ground
    WHEELS_ON = The time point that the aircraft'ss wheels touch on the ground
    TAXI_IN = The time duration elapsed between wheels-on and gate arrival at the destination airport
    CRS_ARR_TIME = Planned arrival time
    ARR_TIME = Actual Arrival Time = ARRIVAL_TIME - SCHEDULED_ARRIVAL
    ARR_DELAY = Total Delay on Arrival in minutes
    CANCELLED = Flight Cancelled (1 = cancelled)
    CANCELLATION_CODE = Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security
    DIVERTED = Aircraft landed on different airport that the one scheduled
    CRS_ELAPSED_TIME = Planned time amount needed for the flight trip
    ACTUAL_ELAPSED_TIME = AIR_TIME+TAXI_IN+TAXI_OUT
    AIR_TIME = The time duration between wheels_off and wheels_on time
    DISTANCE = Distance between two airports
    CARRIER_DELAY = Delay caused by the airline in minutes
    WEATHER_DELAY = Delay caused by weather
    NAS_DELAY = Delay caused by air system
    SECURITY_DELAY = caused by security reasons
    LATE_AIRCRAFT_DELAY = Delay caused by security

### **Note :**
    as you can see , the size of the dataset is too big.
    In this notebook we will clean up the data and make sure to leave the relevant data for the model.

    First, we'll change the OP_CARRIER column which will contain the name of the airlines instead of their code.
    In addition with this column we use to exploring the data set.
    Second, we'll drop all the flights that been canceled 

In [3]:
dataset['OP_CARRIER'].replace({'UA':'United Airlines','AS':'Alaska Airlines','9E':'Endeavor Air','B6':'JetBlue Airways',     
                               'EV':'ExpressJet','F9':'Frontier Airlines','G4':'Allegiant Air','HA':'Hawaiian Airlines',
                               'MQ':'Envoy Air','NK':'Spirit Airlines','OH':'PSA Airlines','OO':'SkyWest Airlines',
                               'VX':'Virgin America','WN':'Southwest Airlines','YV':'Mesa Airline',
                               'YX':'Republic Airways','AA':'American Airlines','DL':'Delta Airlines'} , inplace = True)

dataset = dataset[(dataset['CANCELLED'] == 0)]
## drop OP_CARRIER , and save this column for Exploring the dataset
op_carrier = dataset.pop('OP_CARRIER')
dataset = dataset.drop(columns = ['Unnamed: 27','CANCELLED','CANCELLATION_CODE','DIVERTED','OP_CARRIER_FL_NUM','DEP_TIME', 'ARR_TIME'],axis = 1)

In [4]:
dataset.isna().sum()

FL_DATE                      0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME                 0
DEP_DELAY                 4743
TAXI_OUT                     0
WHEELS_OFF                   0
WHEELS_ON                 2662
TAXI_IN                   2662
CRS_ARR_TIME                 0
ARR_DELAY                20456
CRS_ELAPSED_TIME             7
ACTUAL_ELAPSED_TIME      17858
AIR_TIME                 17858
DISTANCE                     0
CARRIER_DELAY          5744152
WEATHER_DELAY          5744152
NAS_DELAY              5744152
SECURITY_DELAY         5744152
LATE_AIRCRAFT_DELAY    5744152
dtype: int64

    As can be seen in the information above, the following columns:
    CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY : Contains almost 80% percent of empty values.
    (5744152 / 7213446) * 100 = 79.63

In [5]:
dataset = dataset.drop(columns = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'] , axis = 1)
dataset.head(5)

,FL_DATE,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,EWR,DEN,1517,-5.0,15.0,1527.0,1712.0,10.0,1745,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,LAS,SFO,1115,-8.0,11.0,1118.0,1223.0,7.0,1254,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,SNA,DEN,1335,-5.0,15.0,1345.0,1631.0,5.0,1649,-13.0,134.0,126.0,106.0,846.0
3,2018-01-01,RSW,ORD,1546,6.0,19.0,1611.0,1748.0,6.0,1756,-2.0,190.0,182.0,157.0,1120.0
4,2018-01-01,ORD,ALB,630,20.0,13.0,703.0,926.0,10.0,922,14.0,112.0,106.0,83.0,723.0


### **Dealing with Missing Values :**

In [6]:
dataset = dataset.dropna()
print(dataset.isna().sum())
dataset.shape

FL_DATE                0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_DELAY              0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_DELAY              0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
dtype: int64


(7071817, 15)

### **Change the date format :**

In [7]:
dataset['YEAR'] = pd.DatetimeIndex(dataset['FL_DATE']).year
dataset['DAY'] = pd.DatetimeIndex(dataset['FL_DATE']).day
dataset['MONTH'] = pd.DatetimeIndex(dataset['FL_DATE']).month
dataset = dataset.drop(['FL_DATE'],axis = 1)
dataset = dataset[['YEAR','MONTH','DAY','ORIGIN','DEST','CRS_DEP_TIME','DEP_DELAY','TAXI_OUT','WHEELS_OFF','WHEELS_ON','TAXI_IN',
                   'CRS_ARR_TIME','CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME','AIR_TIME','DISTANCE','ARR_DELAY']]
dataset.head(5)

,YEAR,MONTH,DAY,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,ARR_DELAY
0,2018,1,1,EWR,DEN,1517,-5.0,15.0,1527.0,1712.0,10.0,1745,268.0,250.0,225.0,1605.0,-23.0
1,2018,1,1,LAS,SFO,1115,-8.0,11.0,1118.0,1223.0,7.0,1254,99.0,83.0,65.0,414.0,-24.0
2,2018,1,1,SNA,DEN,1335,-5.0,15.0,1345.0,1631.0,5.0,1649,134.0,126.0,106.0,846.0,-13.0
3,2018,1,1,RSW,ORD,1546,6.0,19.0,1611.0,1748.0,6.0,1756,190.0,182.0,157.0,1120.0,-2.0
4,2018,1,1,ORD,ALB,630,20.0,13.0,703.0,926.0,10.0,922,112.0,106.0,83.0,723.0,14.0


### Binary Classification
    Our problem is classification, where a "0" will correspond to a flight being on time, and a "1" to a flight being delayed.

In [8]:
# change ARR_DELAY column: "0" will correspond to a flight being on time, and a "1" to a flight being delayed.
status = []
for value in dataset['ARR_DELAY']:
    if value <= 0:
        status.append(0)
    else:
        status.append(1)
dataset['ARR_DELAY'] = status

In [9]:
dataset.ARR_DELAY.value_counts(normalize = True)

0    0.644384
1    0.355616
Name: ARR_DELAY, dtype: float64

       As you can see  we have highly imbalanced data, as we there are only 35% rows with the value of 1.0 (Delay in flight).
       We will drop a significant amount of rows where our target variable is 0 (No delay in flight).
       And as a result, the size of our datta will be :(5073012,15)

In [10]:
# Split the data into positive and negative
positive_rows = dataset.ARR_DELAY == 1.0
data_pos = dataset.loc[positive_rows]
data_neg = dataset.loc[~positive_rows]

# Merge the balanced data
dataset = pd.concat([data_pos, data_neg.sample(n = len(data_pos))], axis = 0)
# Shuffle the order of data
dataset = dataset.sample(n = len(dataset)).reset_index(drop = True)
dataset.head(5)

,YEAR,MONTH,DAY,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,ARR_DELAY
0,2018,12,20,FLL,MSY,700,-4.0,21.0,717.0,759.0,13.0,806,126.0,136.0,102.0,674.0,1
1,2018,7,23,SFO,ATL,1433,6.0,19.0,1458.0,2225.0,16.0,2222,289.0,302.0,267.0,2139.0,1
2,2018,12,9,MDW,MSP,1743,-4.0,11.0,1750.0,1855.0,8.0,1926,103.0,84.0,65.0,349.0,0
3,2018,3,30,ORD,AUS,1500,-1.0,9.0,1508.0,1725.0,6.0,1747,167.0,152.0,137.0,977.0,0
4,2018,2,17,ELP,DAL,600,-3.0,8.0,605.0,831.0,3.0,840,100.0,97.0,86.0,562.0,0


In [11]:
dataset.ARR_DELAY.value_counts(normalize = True)

0    0.5
1    0.5
Name: ARR_DELAY, dtype: float64

### One Hot encoding
       Because of the fact that Logistic Regression cannot use non-numeric data
       we use one hot encoding with pandas library to convert the non numerical columns to numerical values.
       It is important to mention that the size of the dataset will be significantly larger.
       Thus, the model will work with a data that contains 1% precent of the original dataset. (50297 examples)

In [12]:
dataset = pd.get_dummies(dataset , columns = ['ORIGIN','DEST'])
y_lable = dataset.pop('ARR_DELAY')
dataset = dataset.join(y_lable)
dataset.head(5)

,YEAR,MONTH,DAY,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,...,DEST_VEL,DEST_VLD,DEST_VPS,DEST_WRG,DEST_WYS,DEST_XNA,DEST_YAK,DEST_YNG,DEST_YUM,ARR_DELAY
0,2018,12,20,700,-4.0,21.0,717.0,759.0,13.0,806,...,0,0,0,0,0,0,0,0,0,1
1,2018,7,23,1433,6.0,19.0,1458.0,2225.0,16.0,2222,...,0,0,0,0,0,0,0,0,0,1
2,2018,12,9,1743,-4.0,11.0,1750.0,1855.0,8.0,1926,...,0,0,0,0,0,0,0,0,0,0
3,2018,3,30,1500,-1.0,9.0,1508.0,1725.0,6.0,1747,...,0,0,0,0,0,0,0,0,0,0
4,2018,2,17,600,-3.0,8.0,605.0,831.0,3.0,840,...,0,0,0,0,0,0,0,0,0,0


In [13]:
dataset.shape

(5029704, 731)

In [14]:
newdataset = dataset.sample(frac = 0.01)
newdataset.reset_index()
newdataset.head(5)

,YEAR,MONTH,DAY,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,...,DEST_VEL,DEST_VLD,DEST_VPS,DEST_WRG,DEST_WYS,DEST_XNA,DEST_YAK,DEST_YNG,DEST_YUM,ARR_DELAY
1853661,2018,6,27,1600,3.0,10.0,1613.0,1748.0,8.0,1755,...,0,0,0,0,0,0,0,0,0,1
4500228,2018,8,27,956,-3.0,18.0,1011.0,1636.0,6.0,1657,...,0,0,0,0,0,0,0,0,0,0
1329977,2018,5,6,1635,-1.0,16.0,1650.0,1853.0,6.0,1854,...,0,0,0,0,0,0,0,0,0,1
3257750,2018,3,2,655,-2.0,20.0,713.0,752.0,11.0,803,...,0,0,0,0,0,0,0,0,0,0
436405,2018,12,11,745,16.0,16.0,817.0,1003.0,3.0,957,...,0,0,0,0,0,0,0,0,0,1


In [15]:
## check if the new data set is balanced
newdataset.ARR_DELAY.value_counts(normalize = True)
## save the cleanning data set (newdataset)
newdataset.to_csv("cleanningdata.csv")